# Imports

In [13]:
import requests
import json
import psycopg2
from psycopg2 import connect, Error
from requests.auth import HTTPBasicAuth

# Establishing The Request

In [2]:
header = {'User-Agent': 'PlugShare/6.4.1'}

In [5]:
response = requests.get(
    url = 'https://api.plugshare.com/v3/locations/region?access=1&availability=0%2C1&count=500&include_coming_soon=false&latitude=19.115473015020214&longitude=72.91170636727404&minimal=1&outlets=%5B%7B%22connector%22%3A13%2C%22power%22%3A0%7D%5D&spanLat=30&spanLng=30',
    headers = header,
    auth = HTTPBasicAuth('###','########')
)

In [6]:
response.status_code

200

# Converting The Response To JSON

In [7]:
jsonResponse = response.json()

# Extracting The Values As Per Requirement

In [25]:
responseTata = list(filter(lambda x: 'Tata' or 'tata' in x['name'], jsonResponse))

In [26]:
#succesfully parsed json data to acquire name of all charging stations containing the world Tata
#responseTata contains all the fields which have Tata in name field
type(responseTata)

list

In [27]:
len(responseTata)

218

## Converting The List Object Created Above To A JSON Object

In [28]:
jsonResponseTata = json.dumps(responseTata)

In [29]:
y = json.loads(jsonResponseTata)

In [50]:
y

[{'access': 1,
  'address': '6-81-8, Gajuwaka, Visakhapatnam, Andhra Pradesh 530026, India',
  'icon': 'https://assets.plugshare.com/icons/Y.png',
  'icon_type': 'Y',
  'id': 304992,
  'latitude': 17.684364,
  'longitude': 83.185633,
  'name': 'TML Shiv Shanker Motors Tata Power FC',
  'stations': [{'id': 708115,
    'outlets': [{'connector': 3, 'id': 2135586, 'kilowatts': None, 'power': 0},
     {'connector': 13, 'id': 2135587, 'kilowatts': None, 'power': 0}]}],
  'url': 'http://api.plugshare.com/view/location/304992'},
 {'access': 1,
  'address': '3766, Samantarapur, Old Town, Bhubaneswar, Odisha 751002, India',
  'icon': 'https://assets.plugshare.com/icons/Y.png',
  'icon_type': 'Y',
  'id': 304978,
  'latitude': 20.227008,
  'longitude': 85.84108,
  'name': 'TML Dion Automotives Tata Power FC',
  'stations': [{'id': 708092,
    'outlets': [{'connector': 3, 'id': 2135546, 'kilowatts': None, 'power': 0},
     {'connector': 13, 'id': 2135547, 'kilowatts': None, 'power': 0}]}],
  'url'

## Not relevant to the problem

In [78]:
first_record = responseTata[0]

In [79]:
first_record

{'access': 1,
 'address': '6-81-8, Gajuwaka, Visakhapatnam, Andhra Pradesh 530026, India',
 'icon': 'https://assets.plugshare.com/icons/Y.png',
 'icon_type': 'Y',
 'id': 304992,
 'latitude': 17.684364,
 'longitude': 83.185633,
 'name': 'TML Shiv Shanker Motors Tata Power FC',
 'stations': [{'id': 708115,
   'outlets': [{'connector': 3, 'id': 2135586, 'kilowatts': None, 'power': 0},
    {'connector': 13, 'id': 2135587, 'kilowatts': None, 'power': 0}]}],
 'url': 'http://api.plugshare.com/view/location/304992'}

## Establishing A Connection To The Local Postgres Server

In [30]:
conn = psycopg2.connect(
    database = 'pulse_energy',
    user = 'aaditya',
    password = 'admin',
    host = 'localhost',
    port = '5432'
)

In [31]:
cur = conn.cursor()

In [32]:
cur.execute("ROLLBACK")

# Performing The Query

In [33]:
query = """
    INSERT INTO JSON_DATA VALUES
    (%s, %s, %s, %s)"""
for item in y:
    params = (item['name'], item['address'], item['latitude'], item['longitude'])
    cur.execute(query, params)

In [34]:
conn.commit()

In [36]:
conn.close()

In [35]:
cur.execute("SELECT * FROM json_data")
cur.fetchall()

[('TML Shiv Shanker Motors Tata Power FC',
  '6-81-8, Gajuwaka, Visakhapatnam, Andhra Pradesh 530026, India',
  Decimal('17.684364'),
  Decimal('83.185633')),
 ('TML Shiv Shanker Motors Tata Power FC',
  '6-81-8, Gajuwaka, Visakhapatnam, Andhra Pradesh 530026, India',
  Decimal('17.684364'),
  Decimal('83.185633')),
 ('TML Dion Automotives Tata Power FC',
  '3766, Samantarapur, Old Town, Bhubaneswar, Odisha 751002, India',
  Decimal('20.227008'),
  Decimal('85.84108')),
 ('ASL Motors Tata Power FC',
  'A-7, IInd Phase, Adityapur Industrial Area, Adityapur, Jamshedpur, Jharkhand 832109, India',
  Decimal('22.790721'),
  Decimal('86.147192')),
 ('MG Dehradun Tata Power FC',
  '105, Transport Nagar, Mohabbewala, Dehradun, Uttarakhand 248001, India',
  Decimal('30.268354'),
  Decimal('77.985099')),
 ('Hyway Lounge Tata Power FC',
  'Service Rd, Srinagar Colony, Anantapur, Andhra Pradesh 515002, India',
  Decimal('14.657496'),
  Decimal('77.581425')),
 ('MG Motors Hriday Tata Power FC',
  '

Note that for some reason these changes are not being reflected on my local machine, I am still figuring that out. However as is evident from the above cell, the data is being fetched from the table that resides on my server. The creation of the database was done from the terminal itself and not via the script.